**Making simple VIIRS-DNB annual composites**

In this notebook, we will be conducting a trend analysis on a time series. We shall use VIIRS-Day/Night Bands (DNB) image collection. 

In [12]:
import geemap, ee
import geemap.foliumap as geemap
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


In [13]:
viirs2021 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2021-01-01", "2021-12-31").select('avg_rad')

#there should be 12 images in the collection
print(f"there are {viirs2021.size().getInfo()} images in this collection")

there are 12 images in this collection


In [14]:
#reducing the imagecollection of 12 images to a single image by calculating the median. 
viirs2021med = viirs2021.median()

lat = 14.6091
lon = 121.0223

map1 = geemap.Map(center= [lat,lon], zoom = 9)
map1.add_basemap("SATELLITE")
map1.addLayer(viirs2021med.mask(viirs2021med), {}, "VIIRS-DNB 2019 (monthly median)")
map1.addLayerControl()
map1

**Creating a time series of annual composites from 2019 to 2021**

In [15]:
#using list and map functions to iterate through the ImageCollection and create annual composites for each year. 

start = 2019
end = 2021

years = ee.List.sequence(start,end)

print(f"our list has {years.size().getInfo()} years in it")


our list has 3 years in it


In [16]:
#creating a function to filter images to a given year and reduce them
colID = "NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG"

def viirs_annual_median_reduce(year):
    return ee.ImageCollection(colID).filter(ee.Filter.calendarRange(year, year, "year")).select("avg_rad").median().set('year', year)

yearComps = ee.ImageCollection.fromImages(years.map(viirs_annual_median_reduce))

Map2 = geemap.Map(center = [lat,lon], zoom = 9)
Map2.add_basemap("SATELLITE")
for year in range(start,end+1):
    img = yearComps.filterMetadata("year", "equals", year).first()
    Map2.addLayer(img.mask(img), {}, f"VIIRS-DNB {year}", opacity = 0.75)

In [17]:
Map2.addLayerControl()
Map2

In [20]:
#creating a split panel
left = geemap.ee_tile_layer(yearComps.filterMetadata("year", "equals", 2020), {}, 'VIIRS-DNB 2020', opacity = 0.75)
right = geemap.ee_tile_layer(yearComps.filterMetadata("year", "equals", 2021), {}, 'VIIRS-DNB 2021', opacity = 0.75)

map3= geemap.Map(center = [lat, lon], zoom = 9)
map3.add_basemap("SATELLITE")
map3.split_map(left_layer =left, right_layer =  right)
map3.addLayerControl()
map3

In [22]:
#Legazpi City, Albay
#creating a split panel
left = geemap.ee_tile_layer(yearComps.filterMetadata("year", "equals", 2019), {}, 'VIIRS-DNB 2020', opacity = 0.75)
right = geemap.ee_tile_layer(yearComps.filterMetadata("year", "equals", 2021), {}, 'VIIRS-DNB 2021', opacity = 0.75)


map4= geemap.Map(center = [13.1391, 123.7438], zoom = 9)
map4.add_basemap("SATELLITE")
map4.split_map(left_layer =left, right_layer =  right)
map4.addLayerControl()
map4